In [32]:
#####################
# IMPORT LIBS
#####################

import pandas as pd
import numpy as np
from pathlib import Path
import wandb
import datetime
import os
import random
import joblib

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from lightgbm import LGBMClassifier


#####################
# SET CONSTANTS
#####################

INPUT_PATH = Path('../input')
OUTPUT_PATH = Path('../output')
TRAIN_PATH = INPUT_PATH 

TARGET_COLUMNS = ['sale_flg', 'sale_amount']

RANDOM_SEED = 4444
USE_WANDB = True
CURRENT_TIME = str(datetime.datetime.now()).replace(' ', '_').split('.')[0]

def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(RANDOM_SEED)

In [33]:
###############
# Config
###############



In [34]:
if USE_WANDB:
    
    wandb.login()
    run = wandb.init(project="idao-2021-finals", name = f'{CURRENT_TIME}') # todo add config here

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc
wandb: Currently logged in as: artkulak (use `wandb login --relogin` to force relogin)


In [35]:
%%time

train = pd.read_csv(INPUT_PATH / 'trxn.csv')
assets_under_management = pd.read_csv(INPUT_PATH / 'aum.csv')
balance = pd.read_csv(INPUT_PATH / 'balance.csv')
client = pd.read_csv(INPUT_PATH / 'client.csv')
campaigns = pd.read_csv(INPUT_PATH / 'com.csv')
deals = pd.read_csv(INPUT_PATH / 'deals.csv')
dict_merchant_category_code = pd.read_csv(INPUT_PATH / 'dict_mcc.csv')
payments = pd.read_csv(INPUT_PATH / 'payments.csv')
funnel = pd.read_csv(INPUT_PATH / 'funnel.csv')
appl = pd.read_csv(INPUT_PATH / 'appl.csv')

/opt/conda/lib/python3.7/site-packages/decorator.py:231: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  return caller(func, *(extras + args), **kw)


CPU times: user 6.56 s, sys: 589 ms, total: 7.15 s
Wall time: 7.14 s


## Data prep

In [36]:
X = funnel.drop(columns = TARGET_COLUMNS + ['client_id'])
Y = funnel[TARGET_COLUMNS[0]]

## Train

In [37]:
model = LGBMClassifier(n_jobs = -1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = RANDOM_SEED)
model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
validation_accuracy = accuracy_score(Y_test, Y_pred)
print(f'Accuracy score {validation_accuracy}')

validation_accuracy = accuracy_score(Y_test, Y_pred)

if USE_WANDB:
    run.summary["validation_accuracy"] = validation_accuracy

Accuracy score 0.8506046511627907


NameError: name 'private_test_auc' is not defined

In [ ]:
model.fit(X, Y)

In [ ]:
try:
    os.mkdir(OUTPUT_PATH / 'models')
except:
    pass

# save model
joblib.dump(model, OUTPUT_PATH / 'models' / f'lightgbm_{CURRENT_TIME}.pkl')
# load model
model = joblib.load( OUTPUT_PATH / 'models' / f'lightgbm_{CURRENT_TIME}.pkl')

In [ ]:
if USE_WANDB:
    run